In [1]:
import requests
import concurrent.futures
import csv
from lxml import etree
import re

In [2]:
def scrape_douban(title):
    url1 = "https://book.douban.com/tag/{}".format(title)  # 起始页面url

    def mm(a):
        b = a.split()[1]
        c = b.split('<')[0]
        return c

    page = 0
    while page < 2:  # 遍历页面，page < 10 表示 10 页
        page += 1
        url = url1 + "?start={}&type=T".format(page * 20)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers).text
        tree = etree.HTML(response)
        urls = tree.xpath('//*[@id="subject_list"]/ul/li/div[1]/a/@href')
        #print(urls)
        for url2 in urls:
            response = requests.get(url2, headers=headers).text
            root = etree.HTML(response)
            title = root.xpath('//*[@id="wrapper"]/h1/span/text()')  # 书名
            author = root.xpath('//*[@id="info"]/span[1]/a/text()')  # 作者
            press = root.xpath('//*[@id="info"]/a[1]/text()')  # 出版社
            score = root.xpath('//*[@id="interest_sectl"]/div/div[2]/strong/text()')  # 豆瓣评分
            url = root.xpath('/html/head/meta[10]/@content')  # url
            cover = root.xpath('//*[@id="mainpic"]/a/img/@src')
            time_ = response.selector.re(r'出版年:</span> [0-9]+-[0-9]+-?[0-9]*<br>')[0]
            time = mm(time_)  # 出版日期

            pages = response.selector.re(r'页数:</span> [0-9]+<br>')  # 页数，有些页数没有，赋值为空
            if page == []:
                page = ['0']
                pages = page[0]
            else:
                pages = mm(pages[0])

            price_ = response.selector.re(r'定价:</span> [0-9]+.?[0-9]*[元]?<br>')[0]  # 价格
            price = mm(price_)

            isbn_ = response.selector.re(r'ISBN:</span> [0-9]+<br>')[0]  # ISBN号
            isbn = mm(isbn_)

            a = response.xpath('//*[@id="interest_sectl"]/div/div[2]/div/div[2]/span/a/span/text()')
            if a == []:
                a = ['0']
            evaluators = a[0]  # 评论人数

            books = []
            books.append([title, author, press, score, time, price, isbn, evaluators, cover, url])
    return books

In [4]:
def save_to_csv(books):
    with open('books.csv', 'a+', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for book in books:
            writer.writerow(book)


start_positions = range(0, 200, 20)

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    futures = []
    for start in start_positions:
        futures.append(executor.submit(scrape_douban, '教育'))

    for future in concurrent.futures.as_completed(futures):
        books = future.result()
        save_to_csv(books)

AttributeError: 'str' object has no attribute 'selector'